## Loading packages and data

In [35]:
# Packages
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.stats as st
import seaborn as sns
%tensorflow_version 2.x
import tensorflow
import keras
# import rpy
import datetime
from scipy import stats
# import scipy.cluster.hierarchy as hac
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.model_selection import GridSearchCV
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
# %matplotlib inline
from io import StringIO
import pandas_profiling
from matplotlib import rcParams
from datetime import datetime
import spacy
import nltk
from textblob import TextBlob
import re
nltk.download('wordnet')
nltk.download('punkt')
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
# The charts weren't showing without this in Google Colab
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
sns.set_context('notebook')
%config InlineBackend.figure_format = 'retina'

In [3]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df = pd.read_csv("/content/drive/My Drive/Books, learning materials/NLP/Manning/stackexchange_812k.csv", low_memory=False, index_col=False)

In [14]:
df.head()

,post_id,parent_id,comment_id,text,category
0,1,NaN,NaN,Eliciting priors from experts,title
1,2,NaN,NaN,What is normality?,title
2,3,NaN,NaN,What are some valuable Statistical Analysis op...,title
3,4,NaN,NaN,Assessing the significance of differences in d...,title
4,6,NaN,NaN,The Two Cultures: statistics vs. machine learn...,title


In [18]:
df['parent_id'].isna().sum() # 736597
df['post_id'].isna().sum() # 0
df['comment_id'].isna().sum() # 259056

259056

In [20]:
df['category'].value_counts(normalize=True)

comment    0.681017
post       0.206006
title      0.112977
Name: category, dtype: float64

## Cleaning text

In [0]:
stopwords = list(STOP_WORDS)
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'
# Lemmatizing
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(t):
  t = re.sub('['+my_punctuation + ']+', ' ', str(t)) # remove punctuation
  t = re.sub('\s+', ' ', str(t)) # remove double spaces
      
  # return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(t)] # it will give a list that isn't supported by CountVectorizer
  return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(t)]) # need to keep the result as a line, not a list

In [0]:
# remove everything between the $ signs with nothing: getting rid of formulas 
df['no_formulas'] = df['text'].replace('$.*?$','', regex=True)

In [0]:
df['lemmatized'] = df['no_formulas'].apply(lemmatize_text)

In [0]:
df['lemmatized'] = df['lemmatized'].str.lower() # all lower

In [39]:
df.head()

,post_id,parent_id,comment_id,text,category,lemmatized,no_formulas
0,1,NaN,NaN,Eliciting priors from experts,title,eliciting prior from expert,Eliciting priors from experts
1,2,NaN,NaN,What is normality?,title,what is normality,What is normality?
2,3,NaN,NaN,What are some valuable Statistical Analysis op...,title,what are some valuable statistical analysis op...,What are some valuable Statistical Analysis op...
3,4,NaN,NaN,Assessing the significance of differences in d...,title,assessing the significance of difference in di...,Assessing the significance of differences in d...
4,6,NaN,NaN,The Two Cultures: statistics vs. machine learn...,title,the two cultures statistic v machine learning,The Two Cultures: statistics vs. machine learn...


In [0]:
# Tokenization
def tokenize_text(t):
  t_new = []
  t = nltk.word_tokenize(t)
  for w in t:
    t_new.append(w)
  return t_new 

In [0]:
df['tokens'] = df['lemmatized'].apply(tokenize_text)

In [0]:
df['tokens'] = df['tokens'].apply(lambda tk : ' '.join(tk))

In [64]:
df.head()

,post_id,parent_id,comment_id,text,category,lemmatized,tokens
0,1,NaN,NaN,Eliciting priors from experts,title,eliciting prior from expert,eliciting prior from expert
1,2,NaN,NaN,What is normality?,title,what is normality,what is normality
2,3,NaN,NaN,What are some valuable Statistical Analysis op...,title,what are some valuable statistical analysis op...,what are some valuable statistical analysis op...
3,4,NaN,NaN,Assessing the significance of differences in d...,title,assessing the significance of difference in di...,assessing the significance of difference in di...
4,6,NaN,NaN,The Two Cultures: statistics vs. machine learn...,title,the two cultures statistic v machine learning,the two cultures statistic v machine learning


In [0]:
# dropping no_formulas
df.drop(labels='no_formulas', axis=1, inplace=True)

In [47]:
df.head()

,post_id,parent_id,comment_id,text,category,lemmatized,tokens
0,1,NaN,NaN,Eliciting priors from experts,title,eliciting prior from expert,"[eliciting, prior, from, expert]"
1,2,NaN,NaN,What is normality?,title,what is normality,"[what, is, normality]"
2,3,NaN,NaN,What are some valuable Statistical Analysis op...,title,what are some valuable statistical analysis op...,"[what, are, some, valuable, statistical, analy..."
3,4,NaN,NaN,Assessing the significance of differences in d...,title,assessing the significance of difference in di...,"[assessing, the, significance, of, difference,..."
4,6,NaN,NaN,The Two Cultures: statistics vs. machine learn...,title,the two cultures statistic v machine learning,"[the, two, cultures, statistic, v, machine, le..."


In [0]:
# Saving as a .csv file 
from google.colab import files
df.to_csv("02_25_MK_Manning.csv", index=False)
# files.download("02_25_MK_Manning.csv")

In [0]:
df.to_csv('/content/drive/My Drive/Books, learning materials/NLP/Manning/2_25_mk_manning.csv')